In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hwcc5xiq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hwcc5xiq
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Gohan_test.mp4 to Gohan_test.mp4


In [2]:
import os
import numpy as np
import cv2
from glob import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths for Google Drive folders
DRIVE_VIDEO_DIR = '/content/drive/MyDrive/study_buddy/videos/'
DRIVE_FRAMES_DIR = '/content/drive/MyDrive/study_buddy/frames/'

def create_dir(path):
    try:
        if not os.path.exists(path):
             os.makedirs(path)
    except OSError:
        print(f"Error: creating directory with name {path}")

def save_frame(video_path, save_dir, gap=10):
    name = os.path.basename(video_path).split(".")[0]
    save_path = os.path.join(save_dir, name)
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    idx = 0
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second

    print(f"Processing video: {name}")
    print(f"FPS: {fps}")

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        # Calculate timestamp in seconds
        timestamp_seconds = idx / fps
        # Format timestamp as HH:MM:SS
        hours = int(timestamp_seconds // 3600)
        minutes = int((timestamp_seconds % 3600) // 60)
        seconds = int(timestamp_seconds % 60)
        timestamp = f"{hours:02d}_{minutes:02d}_{seconds:02d}"

        if idx == 0 or idx % gap == 0:
            frame_path = f"{save_path}/frame_{timestamp}.png"
            cv2.imwrite(frame_path, frame)
            if idx % 100 == 0:
                print(f"Saved frame at {timestamp} ({idx} frames processed)")

        idx += 1

    print(f"Completed processing {name}: {idx} total frames, saved frames at every {gap} frames")
    return save_path

if __name__ == "__main__":
    # Get all videos from the Drive folder
    video_paths = glob(os.path.join(DRIVE_VIDEO_DIR, "*.mp4"))
    video_paths.extend(glob(os.path.join(DRIVE_VIDEO_DIR, "*.avi")))
    video_paths.extend(glob(os.path.join(DRIVE_VIDEO_DIR, "*.mov")))

    print(f"Found {len(video_paths)} videos in {DRIVE_VIDEO_DIR}")

    # Process each video
    for path in video_paths:
        print(f"Processing: {path}")
        # Extract a frame every 10 seconds (assuming 30fps)
        frame_gap = int(30 * 10)  # 10 seconds at 30fps
        save_frame(path, DRIVE_FRAMES_DIR, gap=frame_gap)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1 videos in /content/drive/MyDrive/study_buddy/videos/
Processing: /content/drive/MyDrive/study_buddy/videos/Gohan_test.mp4
Processing video: Gohan_test
FPS: 30.0
Saved frame at 00_00_00 (0 frames processed)
Saved frame at 00_00_10 (300 frames processed)
Saved frame at 00_00_20 (600 frames processed)
Saved frame at 00_00_30 (900 frames processed)
Saved frame at 00_00_40 (1200 frames processed)
Saved frame at 00_00_50 (1500 frames processed)
Saved frame at 00_01_00 (1800 frames processed)
Saved frame at 00_01_10 (2100 frames processed)
Saved frame at 00_01_20 (2400 frames processed)
Saved frame at 00_01_30 (2700 frames processed)
Saved frame at 00_01_40 (3000 frames processed)
Saved frame at 00_01_50 (3300 frames processed)
Saved frame at 00_02_00 (3600 frames processed)
Saved frame at 00_02_10 (3900 frames processed)
Saved frame at 00_02_20 (4200 frames

In [3]:
import whisper
import datetime
from google.colab import drive


drive.mount('/content/drive')

# Paths for files in Google Drive
VIDEO_DIR = '/content/drive/MyDrive/study_buddy/videos/'
OUTPUT_DIR = '/content/drive/MyDrive/study_buddy/processed/'

def format_timestamp(seconds):
    return str(datetime.timedelta(seconds=seconds)).split('.')[0]

# Video filename in your Drive
video_filename = 'Gohan_test.mp4'
video_path = f"{VIDEO_DIR}{video_filename}"

# Load the model
model = whisper.load_model('base')

# Transcribe the audio with timestamps
print(f"Transcribing {video_path}...")
result = model.transcribe(video_path, fp16=False)

# Print the transcription with timestamps
print("Transcription with timestamps:")
for segment in result["segments"]:
    start_time = format_timestamp(segment['start'])
    end_time = format_timestamp(segment['end'])
    text = segment['text']

    print(f"[{start_time} --> {end_time}] {text}")

# Save to a file in Google Drive
output_filename = f"{OUTPUT_DIR}{video_filename.split('.')[0]}_transcript.txt"
with open(output_filename, "w") as f:
    for segment in result["segments"]:
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])
        text = segment['text']

        f.write(f"[{start_time} --> {end_time}] {text}\n\n")

print(f"Transcription saved to {output_filename}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 16.5MiB/s]


Transcribing /content/drive/MyDrive/study_buddy/videos/Gohan_test.mp4...
Transcription with timestamps:
[0:00:00 --> 0:00:05]  Correct me if I'm wrong, but did I just hear you threaten to kill me?
[0:00:05 --> 0:00:12]  We don't have to do this. This fight's so it's meaningless. I don't you don't want to fight
[0:00:12 --> 0:00:14]  Yes, I understand that
[0:00:15 --> 0:00:19]  Interesting and what makes you think you can carry out this threat
[0:00:20 --> 0:00:23]  Really I know now what my father meant I
[0:00:24 --> 0:00:28]  Know now why I'm the only one who could take you down
[0:00:30 --> 0:00:32]  So
[0:00:32 --> 0:00:38]  Doesons also S Юr Stalin
[0:00:57 --> 0:01:01]  Yes, I would Summon him
[0:01:01 --> 0:01:08]  They're all you want but I'm helping go on wait for what for sale to rip him to pieces what we watch
[0:01:08 --> 0:01:10]  Wait until he has no choice
[0:01:10 --> 0:01:15]  You're wrong about your son go on may have that power but it doesn't matter
[0:01:15 --> 0:0

" Correct me if I'm wrong, but did I just hear you threaten to kill me? We don't have to do this. This fight's so it's meaningless. I don't you don't want to fight Yes, I understand that Interesting and what makes you think you can carry out this threat Really I know now what my father meant I Know now why I'm the only one who could take you down I Oh Oh, I can't take it. Goku, you can keep standing there all you want, but I'm helping Gohan. Wait. For what? For sale to rip him to pieces while we watch? Wait until he has no choice. You're wrong about your son. Gohan may have that power, but it doesn't matter. He doesn't thirst for battle in Mayhem. He's not a fighter like you. Huh? And so your son may be the most powerful person in the world, but he's also a scared 11-year-old boy. He's not thinking about strength or about competition. He's wondering why his father is standing there, letting him die. Your son's dead Goku. He's dead! He's got to be joking. Giving up already? Well, that's